# RVRP RL Results

In [1]:
import numpy as np
from scipy import stats
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
def compare_routes(n_bins = 30, width = 10, height = 12):
    n_algos = 3
    fig = plt.figure(figsize=(height, width), constrained_layout=True)
    gs0 = fig.add_gridspec(3,2)
    gs00 = gs0[0,0].subgridspec(n_algos,1, hspace=0)
    gs01 = gs0[0,1].subgridspec(n_algos,1, hspace=0)
    gs10 = gs0[1,0].subgridspec(n_algos,1, hspace=0)
    gs11 = gs0[1,1].subgridspec(n_algos,1, hspace=0)
    gs20 = gs0[2,0].subgridspec(n_algos,1, hspace=0)
    gs21 = gs0[2,1].subgridspec(n_algos,1, hspace=0)
    subgrids = [gs00, gs01, gs10, gs11, gs20, gs21]
    axes = []

    routes = [1,2,3,4,5,6]
    ticks = [5, 5, 2, 4, 2.5, 2]
    y_limits = [200, 200, 210, 202, 210, 100]
    
    file_name_RL = './results/RL_results.pickle'
    file_name_DQN = './results/DQN_results.pickle'
    file_name_baseline = './results/baseline_results.pickle'

    for i, route in enumerate(routes):
        with open(file_name_RL, 'rb') as f:
            RL_results = pickle.load(f)
        with open(file_name_DQN, 'rb') as f:
            DQN_results = pickle.load(f)
        with open(file_name_baseline, 'rb') as f:
            rollout_results = pickle.load(f)
        a = RL_results[route]
        b = DQN_results[route]
        c = rollout_results[route]

        # statistics
        mean_a = np.mean(a)
        mean_b = np.mean(b)
        mean_c = np.mean(c)
        var_a = np.var(a)
        var_b = np.var(b)
        var_c = np.var(c)
        _, p_value = stats.ttest_ind(a,b)

        print('ROUTE = {}'.format(route))

        print('RL mean = {}, variance = {}'.format(mean_a, var_a))
        print('DQN mean = {}, variance = {}, p_value = {}'.format(mean_b, var_b, p_value/2))
        
        _, p_value = stats.ttest_ind(a,c)

        print('rollout mean = {}, variance = {}, p_value = {}'.format(mean_c, var_c, p_value/2))

        print('-------------------------------------------')
        
        #  plot
        
        x_max = max(max(a), max(b),max(c))
        bins = np.linspace(0, x_max, n_bins)

        # new_RL_ax 
        axes.append(fig.add_subplot(subgrids[i][0]))
        axes[-1].grid(zorder=0, linestyle='--')
        axes[-1].hist(a, bins, histtype='bar', label = ['rollout'], zorder=3, color = "blue", lw=1)
        axes[-1].set_title('Route {}'.format(route)) 
        axes[-1].set_xticks(np.arange(0,x_max,ticks[i]))
        axes[-1].set_ylim([0, y_limits[i]])
        axes[-1].set_ylabel('Runs')
        axes[-1].set_xticklabels([])
        axes[-1].label_outer()
        axes[-1].legend(prop={'size': 10})
        
        # new_DQN_ax 
        axes.append(fig.add_subplot(subgrids[i][1]))
        axes[-1].grid(zorder=0, linestyle='--')
        _, _, _ = axes[-1].hist(b, bins, histtype='bar', label = ['DQN'], zorder=3, color = "orange", lw=1)
        axes[-1].set_xticks(np.arange(0,x_max,ticks[i]))
        axes[-1].set_ylim([0, y_limits[i]])
        axes[-1].set_ylabel('Runs')
        axes[-1].set_xticklabels([])
        axes[-1].label_outer()
        axes[-1].legend(prop={'size': 10})
        
        # new_B_ax 
        axes.append(fig.add_subplot(subgrids[i][2]))
        axes[-1].grid(zorder=0, linestyle='--')
        _, _, _ = axes[-1].hist(c, bins, histtype='bar', label = ['base policy'], zorder=3, color = "green", lw=1)
        axes[-1].set_xticks(np.arange(0,x_max,ticks[i]))
        axes[-1].set_ylim([0, y_limits[i]])
        axes[-1].set_ylabel('Runs')
        axes[-1].set_xlabel('Cost')
        axes[-1].legend(prop={'size': 10})
        
    fig.tight_layout()
    plt.show()
    fig.savefig('histograms.eps', format='eps')

In [ ]:
compare_routes()